In [12]:
import os
WORK = os.environ["WORK"]
os.chdir(f'{WORK}/ADA_Project')

In [13]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import DeepFAMS

In [14]:
PROJ_DIR = f'{WORK}/ADA_Project'
RAW_IMGS_DIR = f'{PROJ_DIR}/datasets/102flowers_dataset_raw'
RESIZED_IMGS_DIR = f'{PROJ_DIR}/datasets/102flowers_resized'
DATA_CUSTOM_DIR = f'{PROJ_DIR}/datasets/102flowers_custom'
TRAIN_RUNS_DIR = f'{PROJ_DIR}/training_runs/102flowers_training-runs'

In [11]:
# url = 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
# urllib.request.urlretrieve(url, f'{PROJ_DIR}/datasets/102flowers.tgz')

In [ ]:
# tarf = tarfile.open(f'{PROJ_DIR}/datasets/102flowers.tgz')
# tarf.extractall(path=RAW_IMGS_DIR)

In [ ]:
# raw_imgs = glob(f'{RAW_IMGS_DIR}/*')

In [ ]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [ ]:
# DeepFAMS.preprocessing.tf_record_exporter(tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

In [ ]:
# # Needs to be run through the command line at least once to compile the model

# ! module load anaconda && \
#     module load compiler/gcc/4.7 && \
#     module load cuda && \
#     $WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada__source_code/train.py \
#     --outdir=$WORK/ADA_Project/training_runs/102flowers_training-runs \
#     --gpus=2 \
#     --data=$WORK/ADA_Project/datasets/102flowers_custom \
#     --snap=1

In [15]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ada_project/training_runs/102flowers_training-runs/00043-102flowers_custom-auto2-resumecustom/network-snapshot-000008.pkl


In [16]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [17]:
DeepFAMS.utils.execute('nvidia-smi')

Thu Mar 18 12:24:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   32C    P0    37W / 250W |    618MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   

In [18]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.4096
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)